In [73]:
import pandas as pd
import numpy as np
import xgboost as xgb
import sklearn
import time

data = pd.read_parquet('train_data.parquet', engine='pyarrow')

In [74]:

useless_columns = ['target', 'sample_ml_new', 'feature68', 'feature69', 'feature144', 'feature160',
                   'feature292', 'feature406', 'feature407', 'feature496', 'feature511',
                   'feature625', 'feature661', 'feature663', 'feature669', 'feature678',
                   'feature683', 'feature686', 'feature710', 'feature756', 'feature761',
                   'feature765', 'feature801', 'feature802', 'feature806', 'feature807',
                   'feature808', 'feature809', 'feature816', 'feature818', 'feature819',
                   'feature955', 'feature956', 'feature957', 'feature958', 'feature959',
                   'feature960', 'feature961', 'feature962', 'feature963', 'feature964',
                   'feature965', 'feature966', 'feature967', 'feature968', 'feature969',
                   'feature970', 'feature971', 'feature972', 'feature973', 'feature974', 
                   'feature975', 'feature976', 'feature977', 'feature978', 'feature979',
                   'feature980', 'feature981', 'feature982', 'feature983', 'feature984', 
                   'feature1005', 'feature1006', 'feature1007', 'feature1008', 'feature1009',
                   'feature1010', 'feature1011', 'feature1012', 'feature1013', 'feature1014',
                   'feature1015', 'feature1016', 'feature1017', 'feature1018', 'feature1019',
                   'feature1020', 'feature1021', 'feature1022', 'feature1023', 'feature1024',
                   'feature1025', 'feature1026', 'feature1027', 'feature1028', 'feature1029',
                   'feature1030', 'feature1031', 'feature1032', 'feature1033', 'feature1034']

y_data = data["target"]
x_data = data.drop(useless_columns, axis=1)
x_data

,id,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,...,feature1067,feature1068,feature1069,feature1070,feature1071,feature1072,feature1073,feature1074,feature1075,feature1076
0,0,1761,1759,85,105469,191,46,0,6,95,...,28913,48985,84264,12045,12107,12510,21126,28913,48985,84264
1,1,1761,1759,120,105610,144,71,0,135,135,...,0,0,0,0,0,0,0,0,0,0
2,2,890,1759,141,105227,191,11,0,0,0,...,55949,70006,113317,33735,31242,12691,48682,55949,70006,113317
5,5,1599,966,30,102441,191,8,0,0,0,...,0,6872,8530,0,0,0,0,0,6872,8530
6,6,1761,1759,85,104006,191,2,0,0,46,...,0,0,7642,0,0,0,0,0,0,7642
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
693036,693036,1761,1759,141,410,191,2,125,290,298,...,0,0,0,0,0,0,0,0,0,0
693038,693038,1761,1759,0,170,191,1,125,290,298,...,0,0,0,0,0,51714,0,0,0,0
693039,693039,1761,1759,0,168,191,1,125,290,298,...,0,0,0,0,0,51714,0,0,0,0
693040,693040,1761,1759,141,74,191,1,125,290,298,...,0,0,0,0,0,51714,0,0,0,0


In [75]:
X_train, X_test, y_train,  y_test = sklearn.model_selection.train_test_split(x_data, y_data, test_size=0.25, shuffle=True)

X_train_xgb = xgb.DMatrix(X_train, label=y_train)
X_test_xgb = xgb.DMatrix(X_test, label=y_test)


In [80]:
# EQual size of targets

print(y_train.value_counts())
print(y_test.value_counts())


sample_size = 17000 # Задайте желаемое количество строк для каждого класса
frames = []
classes = data['target'].unique()

for i in classes:
    g = data[data['target'] == i].sample(sample_size)
    frames.append(g)

equally_sampled = pd.concat(frames)
equally_sampled['target'].value_counts()

X_train_equally_sampled, X_test_equally_sampled, y_train_equally_sampled,  y_test_equally_sampled = sklearn.model_selection.train_test_split(x_data, y_data, test_size=0.25, shuffle=True)
X_train_xgb_equally_sampled = xgb.DMatrix(X_train_equally_sampled, label=y_train_equally_sampled)
X_test_xgb_equally_sampled = xgb.DMatrix(X_test_equally_sampled, label=y_test_equally_sampled)

target
0    375816
1     13895
Name: count, dtype: int64
target
0    125262
1      4642
Name: count, dtype: int64


In [24]:
xgb_data = xgb.DMatrix(x_data, label=y_data)

In [88]:
params = {'max_depth': 2,
          'eta': 1,
          'objective': "binary:logistic",
          'nthread': -1,
          'eval_metric': ['auc', 'map']}
eval_list = [(X_test_xgb_equally_sampled, "Everything")]
num_rounds = 60

In [92]:
evals_result = dict()
bst = xgb.train(params, X_train_xgb_equally_sampled, num_rounds, eval_list, evals=X_test_xgb_equally_sampled, evals_result=evals_result)

C:\Users\kseni\anaconda3\Lib\site-packages\xgboost\core.py:727: FutureWarning: Pass `evals` as keyword args.
  warnings.warn(msg, FutureWarning)


[0]	Everything-auc:0.63579	Everything-map:0.07591
[1]	Everything-auc:0.66043	Everything-map:0.04938
[2]	Everything-auc:0.68555	Everything-map:0.11745
[3]	Everything-auc:0.69702	Everything-map:0.13389
[4]	Everything-auc:0.70525	Everything-map:0.10769
[5]	Everything-auc:0.71024	Everything-map:0.09089
[6]	Everything-auc:0.71175	Everything-map:0.09287
[7]	Everything-auc:0.71305	Everything-map:0.12047
[8]	Everything-auc:0.71813	Everything-map:0.07252
[9]	Everything-auc:0.71905	Everything-map:0.07839
[10]	Everything-auc:0.72013	Everything-map:0.08299
[11]	Everything-auc:0.72120	Everything-map:0.07114
[12]	Everything-auc:0.72170	Everything-map:0.07853
[13]	Everything-auc:0.72242	Everything-map:0.08925
[14]	Everything-auc:0.72305	Everything-map:0.07113
[15]	Everything-auc:0.72388	Everything-map:0.07044
[16]	Everything-auc:0.72398	Everything-map:0.07885
[17]	Everything-auc:0.72482	Everything-map:0.09023
[18]	Everything-auc:0.72539	Everything-map:0.10015
[19]	Everything-auc:0.72538	Everything-ma

In [93]:

preds = bst.predict(xgb.DMatrix(X_test_equally_sampled))
preds = list(map(round, preds))
y = list(y_test_equally_sampled)

ev = bst.eval(X_test_xgb_equally_sampled)
rec = sklearn.metrics.recall_score(y, preds)
print(ev, ' eval-recall:', rec)

[0]	eval-auc:0.73189570020138750	eval-map:0.15706998001495281  eval-recall: 0.006159728122344945


In [ ]:
data

,id,target,sample_ml_new,feature1,feature2,feature3,feature4,feature5,feature6,feature7,...,feature1067,feature1068,feature1069,feature1070,feature1071,feature1072,feature1073,feature1074,feature1075,feature1076
0,0,0,1,1761,1759,85,105469,191,46,0,...,28913,48985,84264,12045,12107,12510,21126,28913,48985,84264
1,1,0,1,1761,1759,120,105610,144,71,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,1,890,1759,141,105227,191,11,0,...,55949,70006,113317,33735,31242,12691,48682,55949,70006,113317
5,5,0,1,1599,966,30,102441,191,8,0,...,0,6872,8530,0,0,0,0,0,6872,8530
6,6,0,1,1761,1759,85,104006,191,2,0,...,0,0,7642,0,0,0,0,0,0,7642
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
693036,693036,0,1,1761,1759,141,410,191,2,125,...,0,0,0,0,0,0,0,0,0,0
693038,693038,0,1,1761,1759,0,170,191,1,125,...,0,0,0,0,0,51714,0,0,0,0
693039,693039,0,1,1761,1759,0,168,191,1,125,...,0,0,0,0,0,51714,0,0,0,0
693040,693040,0,1,1761,1759,141,74,191,1,125,...,0,0,0,0,0,51714,0,0,0,0


In [67]:
data['feature5'].describe()

count    519615.000000
mean        139.003685
std          74.734122
min           0.000000
25%          63.000000
50%         191.000000
75%         191.000000
max         191.000000
Name: feature5, dtype: float64

In [159]:
data['feature2'].value_counts()

feature2
1759    334027
0         4505
1         3242
2         2660
3         2174
         ...  
1177         7
1708         7
1711         6
1717         6
1518         5
Name: count, Length: 1728, dtype: int64

In [15]:
x=140
corr = data[['target', 'feature'+str(x), 'feature'+str(x+1), 'feature'+str(x+2), 'feature'+str(x+3), 
                'feature'+str(x+4), 'feature'+str(x+5), 'feature'+str(x+6), 'feature'+str(x+7), 'feature'+str(x+8),
                'feature'+str(x+9)]].corr()
corr.style.background_gradient(cmap='coolwarm')

C:\Users\kseni\anaconda3\Lib\site-packages\pandas\io\formats\style.py:3618: RuntimeWarning: All-NaN slice encountered
  smin = np.nanmin(gmap) if vmin is None else vmin
C:\Users\kseni\anaconda3\Lib\site-packages\pandas\io\formats\style.py:3619: RuntimeWarning: All-NaN slice encountered
  smax = np.nanmax(gmap) if vmax is None else vmax


,target,feature140,feature141,feature142,feature143,feature144,feature145,feature146,feature147,feature148,feature149
target,1.000000,0.018013,-0.027897,-0.035972,0.003920,nan,-0.003309,-0.005981,-0.009559,0.010159,0.016216
feature140,0.018013,1.000000,0.074863,0.077542,0.072244,nan,0.089497,-0.003397,0.086332,0.020580,0.032358
feature141,-0.027897,0.074863,1.000000,0.493605,0.122855,nan,0.301292,-0.009977,0.312402,-0.034708,0.014048
feature142,-0.035972,0.077542,0.493605,1.000000,0.154144,nan,0.564539,-0.017306,0.655718,-0.089917,-0.060648
feature143,0.003920,0.072244,0.122855,0.154144,1.000000,nan,0.144641,-0.001349,0.149737,0.016801,0.045139
feature144,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
feature145,-0.003309,0.089497,0.301292,0.564539,0.144641,nan,1.000000,-0.016940,0.886586,0.020851,-0.000943
feature146,-0.005981,-0.003397,-0.009977,-0.017306,-0.001349,nan,-0.016940,1.000000,-0.014700,0.024892,0.005746
feature147,-0.009559,0.086332,0.312402,0.655718,0.149737,nan,0.886586,-0.014700,1.000000,-0.054621,0.002148
feature148,0.010159,0.020580,-0.034708,-0.089917,0.016801,nan,0.020851,0.024892,-0.054621,1.000000,0.026684


In [68]:
boxplot = data.boxplot("feature5")